<h1><font color="#113D68" size=5>Redes neuronales convolucionales</font></h1>



<h1><font color="#113D68" size=6>Caso Práctico: análisis de un problema de clasificación de imágenes con Deep Learning</font></h1>


<br><br>
<div style="text-align: right">
<font size=3>Daniel González</font><br>
<font size=3>IEBS</font>
</div>

---


<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [Caso práctico](#section1)
    - [Parte obligatoria](#section1.1)
    - [Parte opcional](#section1.2)
    - [Objetivos](#section1.3)
    - [Criterios de entrega](#section1.4)
    - [Temporalización](#section1.5)
* [Flickr Style dataset](#section2)
* [Red convolucional desde 0](#section3)
    - [Ejercicio 1](#section3.1)
    - [Ejercicio 2](#section3.2)
* [Red pre-entrenada (InceptionV3)](#section4)
    - [Ejercicio 3](#section4.1)
    - [Ejercicio 4](#section4.2)
    - [Ejercicio 5](#section4.3)
    - [Ejercicio 6](#section4.3)
    - [Ejercicio 7](#section4.4)
    - [Ejercicio 8](#section4.5)
* [Red opcional](#section5)
    - [Ejercicio 7](#section5.1)
* [¿Cuál es el mejor modelo?](#section6)
    - [Ejercicio 8](#section6.1)

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Para mostrar gráficas
import matplotlib.pyplot as plt
%matplotlib inline

# Anaconda fixing problem
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Establecemos una semilla para numpy y tensorflow para poder reproducir la ejecución y los resultados
seed = 101
np.random.seed(seed)
tf.random.set_seed(seed)

# <font color="#004D7F" size=5>Caso práctico</font>

El objetivo de este caso práctico es simular como se haría un análisis completo de un problema de clasificación de imágenes para resolverlo con Deep Learning. Nos pondremos en la piel de un *data scientist* dedicado a analizar y crear modelos de Deep Learning para pasarlos a producción y ser desplegados en una aplicación.

**Destacar que este caso práctico es la continuación de la actividad de la semana 2. En la actividad de la semana 2 utilizamos el modelo pre-entrendo de InceptionV3 y ahora vamos a realizar más experimentos usando redes convolucionales diseñadas por vosotros.**

Imaginemos que tenemos un dataset completo que queremos explotar, nuestra labor será coger este dataset (Flickr Style dataset) y desde 0 intentar llegar a conseguir un modelo que tenga un buen rendimiento ajustándolo poco a poco como hemos visto en clase. Por lo que tendremos que entrerar distintas redes y comparar los resultados que obtengamos en cada experimento para ver cual es mejor.

Cada experimento que tendremos que realizar estará bien definido, la red que deberéis crear y entrenar será proporcinada por lo que solamente tendréis que crear la red que se nos indica con TensorFlow y realizar el entrenamiento de la misma.

## <font color="#004D7F" size=4>Parte obligatoria</font>

Será obligatorio realizar cada uno de los ejercicios que están definidos. En cada ejercicio está definida la red que se tiene que crear y la configuración con la que se tiene que entrenar, por lo que solamente tendréis que pasar esa definición a código con TensorFlow.

Para tener una buena práctica en la realización de este caso práctico se ofrecen esta recomendaciones:

- Utiliza correctamente el sistema de celdas de jupyter. La libreta está realizada de tal forma que solo tendréis que completar las celdas que se indican, ya sea con código o con texto en markdown. Se recomienda rellenar solamente las celdas indicadas para que quede un informe limpio y fácil de seguir. Si fuera necesario incluir más celdas por cualquier motivo se puede hacer pero realizarlo con cuidado para no ensuciar demasiado la libreta.
<br><br>
- Las redes que tendréis que crear en cada experimento son las vistas en clase, por lo que os podéis inspirar en los ejemplos vistos en los tutoriales. Os recomiendo que no copiéis y peguéis código tal cual, sino que lo escribáis por vuestra cuenta y entendáis lo que estáis haciendo en cada momento. Tomaros el tiempo que haga falta para entender cada paso.
<br><br>
- Comprueba que todo se ejecuta correctamente antes de enviar tu trabajo. La mejor forma de enviarlo es exportando la libreta a pdf o html para enviarla en un formato más profesional.

## <font color="#004D7F" size=4>Parte opcional</font>
La parte opcional es el penúltimo ejercicio donde tendréis volver a aplicar la técnica de _fine-tuning_ eligiendo la red pre-entrenada que vosotros queráis y añadiendo las capas que vosotros elijáis.

## <font color="#004D7F" size=4>Objetivos</font>
* Cargar y entender los datos del dataset Flickr Style con los que se trabajarán.
* Crear cada una de las redes indicadas en los experimentos.
* Entrenar cada una de las redes creadas en los experimentos.
* Entender los resultados obtenidos en cada entrenamiento.

## <font color="#004D7F" size=4>Criterios de entrega</font>
Se deberá entregar una libreta de jupyter, aunque se agradecerá que el formato entregado se html o pdf, el trabajo debe estar autocontenido, incluyendo código y texto explicativo para cada sección. 

<a id="section2"></a>
# <font color="#004D7F" size=5>Flickr Style dataset</font>

Este dataset es el que hemos visto en clase y con el que trabajaremos en el caso práctico. Para refrescarlo, es un dataset que contiene imágenes en color donde queremos clasificar cada imagen según el estilo fotográfico al que pertenece.

El dataset de de imágenes Flickr Style tiene las siguintes características:
- Imágenes de 5 tipos de estilo: Detailed, Pastel, Melancholy, Noir y HDR.
- Imágenes en color, es decir, cada pixel tiene 3 valores entre 0 y 255, esos valores corresponden a los valores de RGB (Red, Green, Blue).
- Imágenes de diferentes tamaños, por lo que tendremos que redimensionarlas al mismo tamaño todas antes de usarlas en nuestro modelo.
- 2.000 imágenes en total para el entrenamiento y para el test.

#### **Imporante!!! Solo ejecutar esta celda una sola vez para descargar los datos, no ejecutarlas si ya tenéis los datos descargados**

In [ ]:
!wget 'https://www.dropbox.com/s/ln92e9givhgzugr/flickr_style.zip?dl=0' -O flickr_style.zip
!unzip -q flickr_style.zip
!mkdir data
!mv flickr_style data/

Vamos a cargar los datos de las labels y los datos de las rutas de las imágenes:

In [ ]:
# obtenemos el nombre de las primeras etiquetas seleccionadas
style_label_file = 'data/flickr_style/style_names.txt'
style_labels = list(np.loadtxt(style_label_file, str, delimiter='\n'))
style_labels

In [ ]:
# cargamos los datos de train
train_frame = pd.read_csv('data/flickr_style/train.txt', sep=" ", header=None)
train_frame.columns = ['files','labels_idx']
train_frame['labels'] = train_frame['labels_idx'].map({i:j for i,j in enumerate(style_labels)})

train_frame.head()

In [ ]:
# cargamos los datos de test
test_frame = pd.read_csv('data/flickr_style/test.txt', sep=" ", header=None)
test_frame.columns = ['files','labels_idx']
test_frame['labels'] = test_frame['labels_idx'].map({i:j for i,j in enumerate(style_labels)})

test_frame.head()

In [ ]:
# Mostramos 5 imágenes de cada clase.
plot_n_images = 5
fig = plt.figure(figsize=(20, 25))

np.random.seed(1000)
for i in range(0,5):
    select_frame = train_frame[train_frame['labels_idx']==i]
    for j in range(0,plot_n_images):
        aux_index = np.random.choice(select_frame.index)
        fig_i=fig.add_subplot(plot_n_images,5,j*5+i+1)
        fig_i.imshow(plt.imread(train_frame['files'][aux_index]))
        
        fig_i.set_xticks(())
        fig_i.set_yticks(())
        
    fig_i.set_xlabel('Class %s' % style_labels[i])

<a id="section3"></a>
# <font color="#004D7F" size=5>Red convolucional desde 0</font>

Los primeros experimentos que vamos a realizar será utilizando redes que creemos nosotros mismos. Para ello vamos a tener que transformar las imágense al igual que hicimos en clase, pero esta vez solamente tendremos que realizar estos pasos:

1. Cargar las imágenes
2. Redimensionar todoas las imágenes para tener el mismo tamaño. Usaremos un tamaño de `(150, 150, 3)`.

Como véis, al usar nuestras redes desde 0 no será necesario usar la función `preprocess_input` que tenemos que usar al usar una red pre-entrenada.

La función para transformar las imágenes sería la siguientes:

In [ ]:
def load_img(img_path):
    # cargamos y redimensionamos una imágen
    img = tf.keras.utils.load_img(
        img_path,
        target_size=(150, 150, 3) 
    )
    
    # cambiamos el tipo imagen a un numpy.array
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    
    # normalizamos los valores entre 0 y 1
    return img_array / 255

Ahora vamos a cargar las imágenes para poder entrenar nuestras redes convolucionales:

In [ ]:
# cargamos las imágenes ya transformadas
x_train = np.array([load_img(img_path) for img_path in train_frame['files']])
x_test = np.array([load_img(img_path) for img_path in test_frame['files']])

# cargamos las clases de cada imagen
y_train = train_frame['labels_idx']
y_test = test_frame['labels_idx']

<a id="section3.1"></a>
### <font color="#004D7F" size=4>Ejercicio 1</font>

Crear una red con la siguiente configuración y entrénala:

Arquitectura de la red:

- Capa convolucional `Conv2D` con 16 filtros/kernels de tamaño `(3,3)`, padding con relleno, activación *ReLU* y con entrada `(150,150,3)`
- Capa pooling `MaxPool2D` con reducción de 2 tanto en tamaño como en desplazamiento (stride) y padding con relleno.
- Capa de aplanado `Flatten`.
- Capa densa `Dense` con 64 neuronas y función de activación _ReLU_.
- Capa densa `Dense` con 32 neuronas y función de activación _ReLU_.
- Capa de salida densa `Dense` con 5 neuronas y función de activación _Softmax_.

Configuración del entrenamiento:

- Optimizador: Adam con factor de entrenamiento 0.001
- Función de error: `sparce_categorical_crossentropy`.
- Métricas: `accuracy`.
- Número de _epochs_: 10
- Validation split: 0.2

In [ ]:
# Completar

Evalua el modelo con el conjunto de test y muestra en una gráfica la evolución del entrenamiento:

In [ ]:
# Completar

Escribe un pequeño texto sacando conclusiones de los resultado obtenidos:

COMPLETAR: ESCRIBE AQUÍ TU TEXTO

<a id="section3.2"></a>
### <font color="#004D7F" size=4>Ejercicio 2</font>

Crear una red con la siguiente configuración y entrénala:

Arquitectura de la red:

- Capa convolucional `Conv2D` con 64 filtros/kernels de tamaño `(3,3)`, padding con relleno, activación *ReLU* y con entrada `(150,150,3)`
- Capa pooling `MaxPool2D` con reducción de 2 tanto en tamaño como en desplazamiento (stride) y padding con relleno.
- Capa convolucional `Conv2D` con 32 filtros/kernels de tamaño `(3,3)`, padding con relleno, activación *ReLU*
- Capa pooling `MaxPool2D` con reducción de 2 tanto en tamaño como en desplazamiento (stride) y padding con relleno.
- Capa convolucional `Conv2D` con 32 filtros/kernels de tamaño `(3,3)`, padding con relleno, activación *ReLU*
- Capa pooling `MaxPool2D` con reducción de 2 tanto en tamaño como en desplazamiento (stride) y padding con relleno.
- Capa de aplanado `Flatten`.
- Capa densa `Dense` con 64 neuronas y función de activación _ReLU_.
- Capa `Dropout` con un valor de `0.75`.
- Capa densa `Dense` con 32 neuronas y función de activación _ReLU_.
- Capa densa `Dense` con 32 neuronas y función de activación _ReLU_.
- Capa `Dropout` con un valor de `0.6`.
- Capa de salida densa `Dense` con 10 neuronas y función de activación _Softmax_.

Configuración del entrenamiento:

- Optimizador: Adam con factor de entrenamiento 0.001
- Función de error: `sparce_categorical_crossentropy`.
- Métricas: `accuracy`.
- Número de _epochs_: 50
- Validation split: 0.2

In [ ]:
# Completar

Evalua el modelo con el conjunto de test y muestra en una gráfica la evolución del entrenamiento:

In [ ]:
# Completar

Escribe un pequeño texto sacando conclusiones de los resultado obtenidos:

COMPLETAR: ESCRIBE AQUÍ TU TEXTO

<a id="section4"></a>
# <font color="#004D7F" size=5>Red pre-entrenada (InceptionV3)</font>

Ahora vamos a realizar experimento usando la técnica de _fine-tuning_ y utilizando el modelo pre-entrenado de [_InceptionV3_](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/InceptionV3).

Los ejercicios siguientes podéis observar que son los mismos ejercicios de la actividad de la semana 2.

<a id="section4.1"></a>
## <font color="#004D7F" size=4>Ejercicio 3: preprocesamiento de las imágenes</font>
Como vamos a usar un modelo pre-entreando tenemos que definir la función que nos transforme las imágenes a utilizar. Como hemos visto en clase, para utilizar las imágenes usando un modelo pre-entrenado es necesario realizar una transformación sobre las imágenes que vamos a utilizar, es decir, tenemos que:

1. Cargar las imágenes
2. Redimensionarlas
3. Usar la función de `preprocess_input` del modelo pre-entrenado que vamos a utilizar.

En este caso vamos a utilizar otro modelo pre-entrado entre los que están disponibles en `tf.keras`, esta vez vamos a usar el modelo de _InceptionV3_, um modelo muy popular y usando frecuentemente. Podéis ver toda la información de este modelo [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/InceptionV3). Este modelo utilizar un tamaño de imagen de _**(299, 299, 3)**_, por lo que tendrás que usar este tamaño en el redimensionamiento.

Como hemos hecho en clase, define una función que se llame `load_img_inceptionv3(img_path)` a la cual le pasamos la ruta donde está alojada una imagen y le realiza todas las transformación necesarias para poder se utilizada luego en el proceso de entrenamiento.

In [ ]:
def load_img_inceptionv3(img_path):
    # Completar

**Test**: Puedes probar esta función con el siguiente test, cuando la tengas definida ejecuta la siguiente celda y te debería dar como resultado:

```python
-0.5529412
```

In [ ]:
img = load_img_inceptionv3('data/flickr_style/images/2216312257_2ba4af8439.jpg')
img[0,0,0]

<a id="section4.2"></a>
## <font color="#004D7F" size=4>Ejercicio 4: aplicar el preprocesamiento a todas las imágenes</font>
Una vez tenemos definida nuestra función de para transformar las imágenes, aplíca la transformación tanto al conjunto de train como al conjunto de test como hemos visto en clase.

In [ ]:
# cargamos las imágenes ya transformadas
x_train = ...
x_test = ...

# cargamos las clases de cada imagen
y_train = ...
y_test = ...

**Test**: Puedes probar si lo has hecho correctamente con el siguiente test, cuando hayas terminado ejecuta la siguiente celda y te debería dar como resultado:

```python
(array([-0.5529412, -0.5529412, -0.5372549], dtype=float32), 3)
```

In [ ]:
x_train[0,0,0], y_train[0]

<a id="section4.3"></a>
## <font color="#004D7F" size=4>Ejercicio 5: cargar el modelo pre-entrenado InceptionV3</font>
Una vez tenemos los datos listos, vamos a cargar el modelo pre-entrenado de [_InceptionV3_](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/InceptionV3).

Como vamos a aplicar _fine-tuning_ recuerda usar los siguientes parámetros:
- `input_shape=(299, 299, 3)`
- `include_top=False`
- `pooling='avg'`

Además tienes que congelar todas las capas para que no se entrenen todas, recureda que solo queremos entrenar las últimas que añadamos nosotros.

In [ ]:
base_model = ...

**Test**: Puedes probar si lo has hecho correctamente con el siguiente test, cuando hayas terminado ejecuta la siguiente celda y te debería dar como resultado:

```python
[<keras.layers.merge.Concatenate at **************>,
 <keras.layers.pooling.GlobalAveragePooling2D at **************>]
```

In [ ]:
base_model.layers[-2:]

<a id="section4.4"></a>
## <font color="#004D7F" size=4>Ejercicio 6: añadir capas al modelo (_fine-tuning_)</font>
Una vez tenemos nuestro modelo base, vamos a añadir capas densas al final para entrenarlas y que el modelo se ajuste a nuestros datos.

Añade las siguientes capas al modelo base cargado:
- Capa Dropout con un valor de 0.60.
- Capa Densa de 128 neuronas y función de activación `relu`.
- Capa Dropout con un valor de 0.4.
- Capa Densa de salida con 5 neuronas y función de activación `softmax`.

In [ ]:
# Completar

model = ...

**Test**: Puedes probar si lo has hecho correctamente con el siguiente test, cuando hayas terminado ejecuta la siguiente celda y te debería dar como resultado:

```python
[<keras.layers.core.dropout.Dropout at **************>,
 <keras.layers.core.dense.Dense at **************>,
 <keras.layers.core.dropout.Dropout at **************>,
 <keras.layers.core.dense.Dense at **************>]
```

In [ ]:
model.layers[-4:]

<a id="section4.5"></a>
## <font color="#004D7F" size=4>Ejercicio 7: entrenar el modelo</font>
Una vez tenemos nuestro modelo listo para entrenar, vamos a configurar el entrenamiento y a entrenar nuesto modelo.

En el entrenamiento utiliza:
- Optimizador: Adam con learning rate de 0.001.
- Función de coste: `sparse_categorical_crossentropy`.
- Métricas: `accuracy`.
- Epochs: `25`
- validation_split: 0.2

In [ ]:
# Completar

<a id="section4.6"></a>
## <font color="#004D7F" size=4>Ejercicio 8: evaluar el modelo</font>
Una vez entrenado el modelo usando _fine-tuning_ evalua el modelo usando el conjunto de test en la función `evaluate` y extráe conlcusiones de si el modelo tiene un buen rendimiento o no. Puedes visualizar como ha ido el entrenamiento usando una gráfica como hemos visto en clase.

In [ ]:
# Completar

<a id="section5"></a>
# <font color="#004D7F" size=5>Red opcional</font>

<a id="section5.1"></a>
## <font color="#004D7F" size=4>Ejercicio 7</font>

En este ejercicio tienes vía libre para crear una red que tu creas que va a funcionar mejor. Puedes usar una red construida desde 0 que tu creas que funcionará mejor o puedes usar un modelo pre-entrenado entre los que puedes seleccionar en `tf._keras.applications` que puedes ver [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

Usa las capas que tú quieras y la configuración de entrenamiento que tu elijas.

In [ ]:
# Completar

<a id="section6"></a>
# <font color="#004D7F" size=5>¿Cuál es el mejor modelo?</font>

<a id="section6.1"></a>
## <font color="#004D7F" size=4>Ejercicio 8</font>

Una vez realizado todos los experimentos anteriores, ¿qué modelo elegirías para desplegar en producción? ¿Por qué?

Explica en breves palabras qué modelo eligirías para desplegar en producción y porqué. Compara cada experimento y extráe tus propias conclusiones.

COMPLETAR: ESCRIBE AQUÍ TU TEXTO